# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al; psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell; Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

    CondaPkg Found dependencies: /home/runner/.julia/packages/ASEconvert/CNQ1A/CondaPkg.toml
    CondaPkg Found dependencies: /home/runner/.julia/packages/PythonCall/wXfah/CondaPkg.toml
    CondaPkg Dependencies already up to date


As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123


Python: None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298294081529                   -0.85    5.0         
  2   -8.300181691264       -2.72       -1.25    1.0    135ms
  3   -8.300432799115       -3.60       -1.89    3.1    184ms
  4   -8.300461868913       -4.54       -2.76    1.8    150ms
  5   -8.300464018602       -5.67       -3.03    3.2    315ms
  6   -8.300464322982       -6.52       -3.21    8.0    235ms
  7   -8.300464476175       -6.81       -3.36    2.5    163ms
  8   -8.300464566669       -7.04       -3.52    1.5    134ms
  9   -8.300464620852       -7.27       -3.71    1.6    162ms
 10   -8.300464629916       -8.04       -3.80    2.6    204ms
 11   -8.300464641521       -7.94       -4.06    1.9    158ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.64190218407                   -0.70    5.5         
  2   -16.64605322031       -2.38       -1.14    1.6    347ms
  3   -16.67916476902       -1.48       -1.87    2.4    388ms
  4   -16.67926267625       -4.01       -2.63    4.4    446ms
  5   -16.67928591898       -4.63       -3.04    3.8    475ms
  6   -16.67928611338       -6.71       -3.35    2.4    362ms
  7   -16.67928619912       -7.07       -3.79    3.1    359ms
  8   -16.67928622056       -7.67       -4.52    2.2    375ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32558747090                   -0.56    7.8         
  2   -33.33410212648       -2.07       -1.00    1.1    1.22s
  3   -33.33592400101       -2.74       -1.73    6.1    1.78s
  4   -33.33616180867       -3.62       -2.64    5.5    1.62s
  5   -33.33687981314       -3.14       -2.44    5.2    1.85s
  6   -33.33690391918       -4.62       -2.54    3.6    1.29s
  7   -33.33694350640       -4.40       -3.70    1.9    1.34s
  8   -33.33694373085       -6.65       -3.89    5.6    2.04s
  9   -33.33694378222       -7.29       -4.57    1.6    1.17s


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298593360091                   -0.85    5.2         
  2   -8.300287307667       -2.77       -1.59    1.0    108ms
  3   -8.300435107634       -3.83       -2.57    2.8    146ms
  4   -8.300391723240   +   -4.36       -2.32    3.8    185ms
  5   -8.300464307777       -4.14       -3.48    1.0    111ms
  6   -8.300464598078       -6.54       -3.88    3.0    200ms
  7   -8.300464640820       -7.37       -4.29    1.2    121ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32639216709                   -0.56    7.4         
  2   -33.30611073476   +   -1.69       -1.26    1.2    1.05s
  3   -5.331322989099   +    1.45       -0.34    6.4    2.44s
  4   -33.31776313651        1.45       -1.62    5.6    2.04s
  5   -33.26774506435   +   -1.30       -1.38    4.2    1.68s
  6   -33.29398953647       -1.58       -1.71    2.6    1.41s
  7   -32.96503889174   +   -0.48       -1.28    4.5    1.66s
  8   -33.33456196479       -0.43       -2.34    4.1    1.50s
  9   -33.33635070463       -2.75       -2.44    2.9    1.60s
 10   -33.33660148408       -3.60       -2.71    2.0    1.23s
 11   -33.33679847093       -3.71       -2.90    1.8    1.13s
 12   -33.33687781668       -4.10      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.